# Primitive inputs and outputs

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]==2.0.0rc1
```
</details>

This page gives an overview of the inputs and outputs of the Qiskit Runtime primitives that execute workloads on IBM Quantum&reg; compute resources. These primitives provide you with the ability to efficiently define vectorized workloads by using a data structure known as a **Primitive Unified Bloc (PUB)**. These PUBs are the fundamental unit of work a QPU needs to execute these workloads. They are used as inputs to the [`run()`](../api/qiskit-ibm-runtime/estimator-v2#run) method for the Sampler and Estimator primitives, which execute the defined workload as a job. Then, after the job has completed, the results are returned in a format that is dependent on both the PUBs used as well as the runtime options specified from the Sampler or Estimator primitives.

<span id="pubs"></span>
## Overview of PUBs

When invoking a primitive's [`run()`](../api/qiskit-ibm-runtime/estimator-v2#run) method, the main argument that is required is a `list` of one or more tuples -- one for each circuit being executed by the primitive. Each of these tuples is considered a PUB, and the required elements of each tuple in the list depends on the primitive used. The data provided to these tuples can also be arranged in a variety of shapes to provide flexibility in a workload through broadcasting -- the rules of which are described in a [following section](#broadcasting-rules).

### Estimator PUB
For the Estimator primitive, the format of the PUB should contain at most four values:
- A single `QuantumCircuit`, which may contain one or more [`Parameter`](../api/qiskit/qiskit.circuit.Parameter) objects
- A list of one or more observables, which specify the expectation values to estimate, arranged into an array (for example, a single observable represented as a 0-d array, a list of observables as a 1-d array, and so on). The data can be in any one of the `ObservablesArrayLike` format such as `Pauli`, `SparsePauliOp`, `PauliList`, or `str`.
- A collection of parameter values to bind the circuit against. This can be specified as a single array-like object where the last index is over circuit `Parameter` objects, or omitted (or equivalently, set to `None`) if the circuit has no `Parameter` objects.
- (Optionally) a target precision for expectation values to estimate

### Sampler PUB
For the Sampler primitive, the format of the PUB tuple contains at most three values:
- A single `QuantumCircuit`, which may contain one or more [`Parameter`](../api/qiskit/qiskit.circuit.Parameter) objects
   *Note: These circuits should also include measurement instructions for each of the qubits to be sampled.*
- A collection of parameter values to bind the circuit against $\theta_k$ (only needed if any `Parameter` objects are used that must be bound at runtime)
- (Optionally) a number of shots to measure the circuit with

---

The following code demonstrates an example set of vectorized inputs to the `Estimator` primitive and executes them on an IBM&reg; backend as a single `RuntimeJobV2 ` object.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# Instantiate runtime service and get
# the least busy backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Define a circuit with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.rz(Parameter("b"), 0)
circuit.cx(0, 1)
circuit.h(0)

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
layout = transpiled_circuit.layout


# Now define a sweep over parameter values, the last axis of dimension 2 is
# for the two parameters "a" and "b"
params = np.vstack(
    [
        np.linspace(-np.pi, np.pi, 100),
        np.linspace(-4 * np.pi, 4 * np.pi, 100),
    ]
).T

# Define three observables. The inner length-1 lists cause this array of
# observables to have shape (3, 1), rather than shape (3,) if they were
# omitted.
observables = [
    [SparsePauliOp(["XX", "IY"], [0.5, 0.5])],
    [SparsePauliOp("XX")],
    [SparsePauliOp("IY")],
]
# Apply the same layout as the transpiled circuit.
observables = [
    [observable.apply_layout(layout) for observable in observable_set]
    for observable_set in observables
]

# Estimate the expectation value for all 300 combinations of observables
# and parameter values, where the pub result will have shape (3, 100).
#
# This shape is due to our array of parameter bindings having shape
# (100, 2), combined with our array of observables having shape (3, 1).
estimator_pub = (transpiled_circuit, observables, params)

# Instantiate the new estimator object, then run the transpiled circuit
# using the set of parameters and observables.
estimator = Estimator(mode=backend)
job = estimator.run([estimator_pub])
result = job.result()

### Broadcasting rules

The PUBs aggregate elements from multiple arrays (observables and parameter values) by following the same broadcasting rules as NumPy. This section briefly summarizes those rules.  For a detailed explanation, see the [NumPy broadcasting rules documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html).

Rules:

* Input arrays do not need to have the same number of dimensions.
  * The resulting array will have the same number of dimensions as the input array with the largest dimension.
  * The size of each dimension is the largest size of the corresponding dimension.
  * Missing dimensions are assumed to have size one.
* Shape comparisons start with the rightmost dimension and continue to the left.
* Two dimensions are compatible if their sizes are equal or if one of them is 1.

Examples of array pairs that broadcast:

```text
A1     (1d array):      1
A2     (2d array):  3 x 5
Result (2d array):  3 x 5


A1     (3d array):  11 x 2 x 7
A2     (3d array):  11 x 1 x 7
Result (3d array):  11 x 2 x 7
```

Examples of array pairs that do not broadcast:

```text
A1     (1d array):  5
A2     (1d array):  3

A1     (2d array):      2 x 1
A2     (3d array):  6 x 5 x 4 # This would work if the middle dimension were 2, but it is 5.
```

`EstimatorV2` returns one expectation value estimate for each element of the broadcasted shape.

Here are some examples of common patterns expressed in terms of array broadcasting.  Their accompanying visual representation is shown in the figure that follows:


Parameter value sets are represented by n x m arrays, and observable arrays are represented by one or more single-column arrays. For each example in the previous code, the parameter value sets are combined with their observable array to create the resulting expectation value estimates.  Example 1 (broadcast single observable) has a parameter value set that is a 5x1 array and a 1x1 observables array.  The one item in the observables array is combined with each item in the parameter value set to create a single 5x1 array where each item is a combination of the original item in the parameter value set with the item in the observables array.  Example 2 (zip) has a 5x1 parameter value set and a 5x1 observables array.  The output is a 5x1 array where each item is a combination of the nth item in the parameter value set with the nth item in the observables array.  Example 3 (outer/product) has a 1x6 parameter value set and a 4x1 observables array.  Their combination results in a 4x6 array that is created by combining each item in the parameter value set with *every* item in the observables array, and thus each parameter value becomes an entire column in the output.  Example 4 (Standard nd generalization) has a 3x6 parameter value set array and two 3x1 observables array.  These combine to create two 3x6 output arrays in a similar manner to the previous example.

![This image illustrates several visual representations of array broadcasting](/images/guides/broadcasting.svg "Visual representation of broadcasting")

In [2]:
# Broadcast single observable


parameter_values = np.random.uniform(size=(5,))  # shape (5,)
observables = SparsePauliOp("ZZZ")  # shape ()
# >> pub result has shape (5,)

# Zip
parameter_values = np.random.uniform(size=(5,))  # shape (5,)
observables = [
    SparsePauliOp(pauli) for pauli in ["III", "XXX", "YYY", "ZZZ", "XYZ"]
]  # shape (5,)
# >> pub result has shape (5,)

# Outer/Product
parameter_values = np.random.uniform(size=(1, 6))  # shape (1, 6)
observables = [
    [SparsePauliOp(pauli)] for pauli in ["III", "XXX", "YYY", "ZZZ"]
]  # shape (4, 1)
# >> pub result has shape (4, 6)

# Standard nd generalization
parameter_values = np.random.uniform(size=(3, 6))  # shape (3, 6)
observables = [
    [
        [SparsePauliOp(["XII"])],
        [SparsePauliOp(["IXI"])],
        [SparsePauliOp(["IIX"])],
    ],
    [
        [SparsePauliOp(["ZII"])],
        [SparsePauliOp(["IZI"])],
        [SparsePauliOp(["IIZ"])],
    ],
]  # shape (2, 3, 1)

<Admonition type="tip" title="SparsePauliOp">
Each `SparsePauliOp` counts as a single element in this context, regardless of the number of Paulis contained in the `SparsePauliOp`. Thus, for the purpose of these broadcasting rules, all of the following elements have the same shape:

```text
a = SparsePauliOp("Z") # shape ()
b = SparsePauliOp("IIIIZXYIZ") # shape ()
c = SparsePauliOp.from_list(["XX", "XY", "IZ"]) # shape ()
```

The following lists of operators, while equivalent in terms of information contained, have different shapes:

```text
list1 = SparsePauliOp.from_list(["XX", "XY", "IZ"]) # shape ()
list2 = [SparsePauliOp("XX"), SparsePauliOp("XY"), SparsePauliOp("IZ")] # shape (3, )
```
</Admonition>

## Overview of primitive results

Once one or more PUBs are sent to a QPU for execution and a job successfully completes, the data is returned as a [`PrimitiveResult`](../api/qiskit/qiskit.primitives.PrimitiveResult) container object accessed by calling the `RuntimeJobV2.result()` method. The `PrimitiveResult` contains an iterable list of [`PubResult`](../api/qiskit/qiskit.primitives.PubResult) objects that contain the execution results for each PUB. Depending on the primitive used, these data will be either expectation values and their error bars in the case of the Estimator, or samples of the circuit output in the case of the Sampler.

Each element of this list corresponds to each PUB submitted to the primitive's `run()` method (for example, a job submitted with 20 PUBs will return a `PrimitiveResult` object that contains a list of 20 `PubResults`, one corresponding to each PUB).

Each of these `PubResult` objects possess both a `data` and a `metadata` attribute. The `data` attribute is a customized [`DataBin`](../api/qiskit/qiskit.primitives.DataBin) that contains the actual measurement values, standard deviations, and so forth. This `DataBin` has various attributes depending on the shape or structure of the associated PUB as well as the error mitigation options specified by the primitive used to submit the job (for example, [ZNE](./error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) or [PEC](./error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec)). Meanwhile, the `metadata` attribute contains information about the runtime and error mitigation options used (explained later in the [Result metadata](#result-metadata) section of this page).

Put simply, a single job returns a `PrimitiveResult` object and contains a list of one or more `PubResults`. These `PubResults` are where the measurement data is stored for each of the PUBs that were submitted to the job.

### Estimator output

Each `PubResult` for the Estimator primitive contains at least an array of expectation values (`PubResult.data.evs`) and associated standard deviations (either `PubResult.data.stds` or `PubResult.data.ensemble_standard_error` depending on the `resilience_level` used), but can contain more data depending on the error mitigation options that were specified.

The below code snippet describes the `PrimitiveResult` (and associated `PubResult`) format for the job created above.

In [3]:
print(
    f"The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n"
)
print(
    f"The associated PubResult of this job has the following DataBins:\n {result[0].data}\n"
)
print(f"And this DataBin has attributes: {result[0].data.keys()}")
print(
    "Recall that this shape is due to our array of parameter binding sets having shape (100,), combined with \n\
  our array of observables having shape (3, 1), where 2 is the number of parameters in the circuit.\n"
)
print(
    f"The expectation values measured from this PUB are: \n{result[0].data.evs}"
)

The result of the submitted job had 1 PUB and has a value:
 PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(3, 100), dtype=float64>), stds=np.ndarray(<shape=(3, 100), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(3, 100), dtype=float64>), shape=(3, 100)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})

The associated PubResult of this job has the following DataBins:
 DataBin(evs=np.ndarray(<shape=(3, 100), dtype=float64>), stds=np.ndarray(<shape=(

#### How the Estimator calculates error

In addition to the estimate of the mean of the observables passed in the input PUBs (the `evs` field of the `DataBin`), the Estimator also attempts to deliver an estimate of the error associated with those expectation values. All estimator queries will populate the `stds` field with a quantity like the standard error of the mean for each expectation value, but some error mitigation options produce additional information, such as `ensemble_standard_error`.

Consider a single observable $\mathcal{O}$. In the absence of [ZNE](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne), you can think of each shot of the Estimator execution as providing a point estimate of the expectation value $\langle \mathcal{O} \rangle$. If the pointwise estimates are in a vector `Os`, then the value returned in `ensemble_standard_error` is equivalent to the following (in which $\sigma_{\mathcal{O}}$ is the standard deviation of the expectation value estimate and $N_{shots}$ is the number of shots):

$$\frac{ \sigma_{\mathcal{O}} }{ \sqrt{N_{shots}} },$$

which treats all shots as part of a single ensemble. If you requested gate [twirling](/guides/error-mitigation-and-suppression-techniques#pauli-twirling) (`twirling.enable_gates = True`), you can sort the pointwise estimates of $\langle \mathcal{O} \rangle$ into sets that share a common twirl. Call these sets of estimates `O_twirls`, and there are `num_randomizations` (number of twirls) of them. Then `stds` is the standard error of the mean of `O_twirls`, as in

$$\frac{ \sigma_{\mathcal{O}} }{ \sqrt{N_{twirls}} },$$

where $\sigma_{\mathcal{O}}$ is the standard deviation of `O_twirls` and $N_{twirls}$ is the number of twirls. When you do not enable twirling, `stds` and `ensemble_standard_error` are equal.

If you enable ZNE, then the `stds` described above become weights in a non-linear regression to an extrapolator model. What finally gets returned in the `stds` in this case is the uncertainty of the fit model evaluated at a noise factor of zero. When there is a poor fit, or large uncertainty in the fit, the reported `stds` can become very large. When ZNE is enabled, `pub_result.data.evs_noise_factors` and `pub_result.data.stds_noise_factors` are also populated, so that you can do your own extrapolation.

### Sampler output


The Sampler primitive outputs job results in a similar format, with the exception that each `DataBin` will contain one or more `BitArray` objects which store the samples of the circuit attached to a particular `ClassicalRegister`, typically one bitstring per shot. The attribute label for each bit array object depends on the `ClassicalRegisters` defined in the circuit being executed. The measurement data from these `BitArrays` can then be processed into a dictionary with key-value pairs corresponding to each bitstring measured (for example, '1011001') and the number of times (or counts) it was measured.

For example, a circuit that has measurement instructions added by the [`QuantumCircuit.measure_all()`](../api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) function possesses a classical register with the label *'meas'*. After execution, a count data dictionary can be created by executing:

In [4]:
# Add measurement instructions to the example circuit
circuit.measure_all()

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)

# Create a PUB for the Sampler primitive using the same parameters defined earlier
sampler_pub = (transpiled_circuit, params)


sampler = Sampler(mode=backend)
job = sampler.run([sampler_pub])
result = job.result()

In [5]:
print(
    f"The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n"
)
print(
    f"The associated PubResult of this Sampler job has the following DataBins:\n {result[0].data}\n"
)
print(f"It has a key-value pair dict: \n{result[0].data.items()}\n")
print(
    f"And the raw data can be converted to a bitstring-count format: \n{result[0].data.meas.get_counts()}"
)

The result of the submitted job had 1 PUB and has a value:
 PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(100,), num_shots=4096, num_bits=2>), shape=(100,)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-03-13 17:37:03', stop='2025-03-13 17:38:48', size=409600>)])}, 'version': 2})

The associated PubResult of this Sampler job has the following DataBins:
 DataBin(meas=BitArray(<shape=(100,), num_shots=4096, num_bits=2>), shape=(100,))

It has a key-value pair dict: 
dict_items([('meas', BitArray(<shape=(100,), num_shots=4096, num_bits=2>))])



And the raw data can be converted to a bitstring-count format: 
{'11': 106114, '10': 101385, '01': 100670, '00': 101431}


## Result metadata

In addition to the execution results, both the `PrimitiveResult` and `PubResult` objects contain a metadata attribute about the job that was submitted. The metadata containing information for all submitted PUBs (such as the various [runtime options](../api/qiskit-ibm-runtime/options) available) can be found in the `PrimitiveResult.metatada`, while the metadata specific to each PUB is found in `PubResult.metadata`.

In [6]:
# Print out the results metadata
print("The metadata of the PrimitiveResult is:")
for key, val in result.metadata.items():
    print(f"'{key}' : {val},")

print("\nThe metadata of the PubResult result is:")
for key, val in result[0].metadata.items():
    print(f"'{key}' : {val},")

The metadata of the PrimitiveResult is:
'execution' : {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-03-13 17:37:03', stop='2025-03-13 17:38:48', size=409600>)])},
'version' : 2,

The metadata of the PubResult result is:
'circuit_metadata' : {},


For Sampler jobs, you can also review the result metadata to understand when certain data was run; this is called the [_execution span._](monitor-job#execution-spans)